1. 홈플러스 url = www.homeplus.co.kr/
2. 액션 연속 취하기

In [18]:
from datetime import datetime
datetime.today().strftime("%Y%m%d")

'20200421'

In [20]:
# 라이브러리 선언
import bs4,time # 메일 전송 라이브러리, 문자를 안보이게하는 라이브러리
import pandas as pd
from selenium import webdriver
from email.message import EmailMessage
from selenium.webdriver.common.action_chains import ActionChains
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}
# data=requests.get(url,headers=headers)

# 크롬드라이버 위치 및 드라이버 생성
driver_loc = "./chromedriver/chromedriver.exe"
options = webdriver.ChromeOptions()
options.add_argument("window-size = 1920x1080")

driver = webdriver.Chrome(driver_loc, options=options)

# 접속할 사이트
url = "http://www.homeplus.co.kr/app.exhibition.main.Main.ghs?paper_no=category"

# driver를 이용해 사이트 접속
driver.get(url)
time.sleep(1)
# print(driver.get_cookies())

# Xpath
mainMenu = '//*[@id="ui-menu-catgnb"]' # 카테고리 창
subMenu = '//*[@id="ui-catlist-tab1"]/ul[1]/li[5]/a' # 정육코너

main = driver.find_element_by_xpath(mainMenu)
sub = driver.find_element_by_xpath(subMenu)
popularList = '//*[@id="bestHotProduct"]'
# 메인 카테고리 창 이동 후 클릭
ActionChains(driver).move_to_element(main).click(sub).perform()
time.sleep(1)
driver.find_element_by_xpath(popularList).click()

time.sleep(3)      # 페이지가 로딩될 때까지 3초기다림 -> 이 부분을 안해주었더니 계속 소스가 짤려서 들어옴

# 페이지 소스를 저장
data = driver.page_source


# 텍스트화된 데이터를 html형태로 추출
bs = bs4.BeautifulSoup(data,'html.parser')
divTags = bs.findAll(name = 'div',
          attrs = {"class":"exh-comtxt w-ty4"})
rows=[]
i = 0
for i in range(0, len(divTags)):
    price = divTags[i].find(name = 'span',
                   attrs = {'class':'unit'}).text.replace("\t","")
    title = divTags[i].find(name = 'a',
                   attrs = {'class':'name'}).text
    rows.append([price,title])
    
result = pd.DataFrame(data = rows, columns=['가격','상품명'])
from datetime import datetime
time = datetime.today().strftime("%Y%m%d")
file = './'+ time +".csv"
result.to_csv(file, encoding = 'ms949', index = False)

In [9]:
import requests, bs4
import pandas as pd

# 웹페이지 요청하기
resp= requests.get(url = "http://www.homeplus.co.kr/app.exhibition.category.Category.ghs?comm=category.list&cid=60126&WT.ac=GNBctg_1dep_ctg")

# 요청한 웹페이지 텍스트화 
html = resp.text

# utf-8로 한글깨짐 방지
resp.encoding = 'utf-8'


In [10]:
# bs4로 텍스트화한 파일을 html형태로 parser하기
bs = bs4.BeautifulSoup(html, 'html.parser' )
bs

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<meta content="" name="description"/>
<title>홈플러스 카테고리 대</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="no" http-equiv="imagetoolbar"/>
<meta content="홈플러스 온라인" property="og:site_name"/>
<meta content="신선, 가공식품, 유아, 생활, 주방용품, 행사 정보 제공, 당일 배송" property="og:keywords"/>
<link href="http://www.homeplus.co.kr/GHS/images/mart/etc/favicon_homeplus.png" rel="shortcut icon"/><!-- 2018-10-17 : 추가 : favicon 추가 -->
<link href="http://www.homeplus.co.kr/GHS/images/mart/etc/favicon.ico?2018102901" rel="shortcut icon"/><!-- 2018-10-17 : 추가 : favicon 추가 -->
<link href="/GHS/mart/common/css/common.css?v=2020040101" rel="stylesheet"/>
<link href="/GHS/mart/common/css/layout.css?v=2020032501" rel="stylesheet"/>


In [ ]:
# 태그 이름이 div이고 속성 class 이름이 class_spark_quiz 찾기
divtag = bs.find(name = "div",
                attrs = {"class" : "class_spark_quiz"})

# divtag에서 태그 'a' 찾기
atags = divtag.findAll(name = "a", limit = 2)

# 컬럼을 담을 리스트를 선언함
totalList = []

# 반복문으로 리스트에 담기
for i in range(0, len(atags)):
    
    # atags에서 속성 'href'의 값을 eachLink에 저장
    eachLink = atags[i].attrs["href"]
    # 텍스트를 eachText에 저장
    eachText = atags[i].text
    # 리스트의 형태로 totalList에 추가 [[a, b],[c, d], ...]
    totalList.append([eachLink, eachText])

# 컬럼명이 "링크", "타이틀" 인 데이터프레임으로 저장
result = pd.DataFrame(data = totalList, columns = ["링크", "타이틀"])
result